In [1]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import torchvision
from torchvision import datasets,transforms
import torch.optim as optim

$example$

In [2]:
logits = torch.rand(4,10)

In [3]:
pred = F.softmax(logits,dim = 1)

In [4]:
pred.shape

torch.Size([4, 10])

In [5]:
pred_label = pred.argmax(dim = 1)

In [6]:
pred_label

tensor([5, 5, 3, 5])

In [7]:
logits.argmax(dim = 1)

tensor([5, 5, 3, 5])

In [8]:
label = torch.tensor([5,4,3,9])
correct = torch.eq(pred_label,label)

In [9]:
correct

tensor([ True, False,  True, False])

In [10]:
correct.sum().float().sum()/4

tensor(0.5000)

### $When\space to\space test$

- $test\space once\space per \space several \space batch$

- $test\space once\space per \space epoch$

- $epoch\space V.S.\space step$

In [11]:
learning_rate = 1e-2
batch_size = 64
epochs = 10

In [12]:
train_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),                       # 数据类型转化
                torchvision.transforms.Normalize((0.1307, ), (0.3081, )) # 数据归一化处理
    ])), batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(torchvision.datasets.MNIST('datasets/mnist_data/',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307, ), (0.3081, ))
    ])),batch_size=batch_size,shuffle=False)

In [13]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784,200),
            nn.LeakyReLU(inplace = True),
            nn.Linear(200,200),
            nn.LeakyReLU(inplace = True),
            nn.Linear(200,10),
            nn.LeakyReLU(inplace = True)
        )
        
    def forward(self,x):
        x = self.model(x)
        
        return x

In [14]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(),lr = learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

In [15]:
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        data = data.view(-1,28*28)
        data,target = data.to(device),target.to(device)
        
        logits = net(data)
        loss = criteon(logits,target)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.300152
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.038405
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.609878
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.948645
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.578279
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.397320
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.392897
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.433338
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.350583
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.291713

Test set: Average loss: 0.0053, Accuracy: 9023/10000 (90%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.703412
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.318359
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.486072
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.137833
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.274568
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.239691
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.116818
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.272905
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.426216
T